In [5]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import dlib

In [6]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("./models/shape_predictor_68_face_landmarks.dat")

In [7]:
cap = cv2.VideoCapture(0)
ALERT_DURATION = 15
closed_eyes_counter = 0
threshold = 120  
alert_threshold = 15

In [8]:
model = load_model('drowsiness_model.h5')

def extract_eye_roi(gray, landmarks, eye_points):
    eye = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in eye_points])
    x, y, w, h = cv2.boundingRect(eye)
    roi = gray[y:y+h, x:x+w]
    return roi

LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    eye_open = False
    focus_status = "Not Focused"

    for face in faces:
        landmarks = landmark_predictor(gray, face)

        for eye_points in [LEFT_EYE_POINTS, RIGHT_EYE_POINTS]:
            eye_roi = extract_eye_roi(gray, landmarks, eye_points)

            if eye_roi.size == 0 or eye_roi.shape[0] < 10 or eye_roi.shape[1] < 10:
                continue

            eye_roi_resized = cv2.resize(eye_roi, (48, 48))
            eye_roi_resized = eye_roi_resized.astype("float32") / 255.0
            eye_roi_resized = np.expand_dims(eye_roi_resized, axis=-1)
            eye_roi_resized = np.expand_dims(eye_roi_resized, axis=0)

            prediction = model.predict(eye_roi_resized, verbose=0)[0]
            print(prediction)
            open_prob = prediction[1]

            if open_prob > 0.6:
                eye_open = True
                break  # One open eye is enough

        # Draw rectangle around face
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()),
                      (0, 255, 0) if eye_open else (0, 0, 255), 2)

    if eye_open:
        focus_status = "Focused"
        closed_eyes_counter = 0
    else:
        closed_eyes_counter += 1

    cv2.putText(frame, f"Focus: {focus_status}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if focus_status == "Focused" else (0, 0, 255), 2)

    if closed_eyes_counter >= ALERT_DURATION:
        cv2.putText(frame, "DROWSINESS ALERT!", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Drowsiness Detector (Dlib)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[0.99374336 0.00625668]
[0.76135385 0.23864615]
[0.811893 0.188107]
[0.89628375 0.10371624]
[0.0192389 0.9807612]
[0.1641349 0.8358651]
[0.11297932 0.8870207 ]
[0.12077869 0.87922126]
[0.95861095 0.04138899]
[0.99424815 0.00575181]
[0.9367286 0.0632714]
[0.03946873 0.9605313 ]
[0.2581717 0.7418283]
[0.9920556  0.00794444]
[1.0000000e+00 2.4482373e-08]
[0.41302654 0.5869735 ]
[9.999987e-01 1.353234e-06]
[0.09425613 0.90574384]
[0.09271605 0.90728396]
[0.95819485 0.04180516]
[0.9986197  0.00138035]
[0.12577148 0.8742285 ]
[0.45042676 0.54957324]
[1.0000000e+00 1.1593476e-08]
[9.9999869e-01 1.3226797e-06]
[9.9999988e-01 1.6951402e-07]
[9.9920624e-01 7.9377636e-04]
[1.000000e+00 7.647955e-09]
[0.7209226  0.27907738]
[0.9331116  0.06688841]
[0.43344718 0.56655276]
[0.6970446  0.30295533]
[0.86271745 0.13728255]
[0.6238494 0.3761506]
[0.01173582 0.9882642 ]
[0.8100295  0.18997052]
[0.94243443 0.05756557]
[0.9070816  0.09291843]
[0.92253155 0.07746848]
[0.7391133  0.26088676]
[0.82367414 0.17